https://towardsdatascience.com/train-a-gpt-2-transformer-to-write-harry-potter-books-edf8b2e3f3db

Preprocessing the text

In [6]:
import os
import re

In [7]:
pip install transformers

In [8]:
%%bash
wget https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/GPT2-HarryPotter-Training/data_cleanup/merged_books.txt

--2020-10-13 07:57:31--  https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/GPT2-HarryPotter-Training/data_cleanup/merged_books.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2972019 (2.8M) [text/plain]
Saving to: ‘merged_books.txt’

     0K .......... .......... .......... .......... ..........  1% 1.40M 2s
    50K .......... .......... .......... .......... ..........  3% 3.14M 1s
   100K .......... .......... .......... .......... ..........  5% 7.71M 1s
   150K .......... .......... .......... .......... ..........  6% 11.2M 1s
   200K .......... .......... .......... .......... ..........  8% 3.95M 1s
   250K .......... .......... .......... .......... .......... 10% 7.09M 1s
   300K .......... .......... .......... .......... ........

In [9]:
with open("merged_books.txt",'r') as f:
    string = f.read()
    print(string)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
def clean_text(string):
    pattern = '(page|PAGE|Page)(\s+\|\s+)([0-9]+)(.*)$'
    output_cleaned = re.sub('\s$', '', string, flags=re.MULTILINE)
    p=re.compile(pattern,re.MULTILINE)
    output_cleaned = p.sub(" ",output_cleaned)
    return output_cleaned

In [11]:
check = clean_text(string)

In [12]:
check

Output hidden; open in https://colab.research.google.com to view.

In [13]:
with open("output_cleaned.txt",'w') as f:
     f.writelines(check)

In [14]:
train_doc =[]
val_doc =[]

In [15]:
with open("output_cleaned.txt",'r') as f:
    file_input=f.readlines()

In [16]:
file_input

['\n',
 'THE BOY WHO LIVED\n',
 'Mr. and Mrs. Dursley, of number four, Privet Drive,\n',
 'were proud to say that they were perfectly normal,\n',
 'thank you very much. They were the last people you’d\n',
 'expect to be involved in anything strange or\n',
 'mysterious, because they just didn’t hold with such\n',
 'nonsense.\n',
 'Mr. Dursley was the director of a firm called\n',
 'Grunnings, which made drills. He was a big, beefy\n',
 'man with hardly any neck, although he did have a\n',
 'very large mustache. Mrs. Dursley was thin and\n',
 'blonde and had nearly twice the usual amount of\n',
 'neck, which came in very useful as she spent so\n',
 'much of her time craning over garden fences, spying\n',
 'on the neighbors. The Dursley s had a small son\n',
 'called Dudley and in their opinion there was no finer\n',
 'boy anywhere.\n',
 'The Dursleys had everything they wanted, but they\n',
 'also had a secret, and their greatest fear was that\n',
 'somebody would discover it. They didn’

In [17]:
count = 0
for cnt, line in enumerate(file_input):
#         print(cnt)
        if cnt <= len(file_input)*0.90:
            train_doc.append(line)
        else:
            val_doc.append(line)

In [18]:
len(train_doc)

54937

In [19]:
## Write to file
f = open('train_harry.txt', "w+")
count = 0
for line in train_doc:
    count=count+1
    f.write(str(line))
    f.write("\n")
    
f.close()
print(count)

54937


In [20]:
## Write to file
f = open('val_harry.txt', "w+")
count = 0
for line in val_doc:
    count=count+1
    f.write(str(line))
    f.write("\n")
    
f.close()
print(count)

6103


In [21]:
!wget https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/GPT2-HarryPotter-Training/examples/run_lm_finetuning.py

--2020-10-13 07:58:16--  https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/GPT2-HarryPotter-Training/examples/run_lm_finetuning.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31104 (30K) [text/plain]
Saving to: ‘run_lm_finetuning.py’

run_lm_finetuning.p 100%[===================>]  30.38K  --.-KB/s    in 0.02s   

2020-10-13 07:58:16 (1.59 MB/s) - ‘run_lm_finetuning.py’ saved [31104/31104]



In [22]:
%%bash
export TRAIN_FILE=train_harry.txt
export TEST_FILE=val_harry.txt
export MODEL_NAME=gpt2
export MODEL_NAME_OR_PATH=gpt2-medium
export OUTPUT_DIR=output

python run_lm_finetuning.py \
    --output_dir=$OUTPUT_DIR \
    --model_type=$MODEL_NAME \
    --model_name_or_path=$MODEL_NAME_OR_PATH \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --overwrite_output_dir\
    --block_size=200\
    --per_gpu_train_batch_size=1\
    --save_steps 5000\
    --num_train_epochs=2

2020-10-13 07:58:20.898445: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/13/2020 07:58:22 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
10/13/2020 07:58:23 - INFO - filelock -   Lock 140696551537464 acquired on /root/.cache/torch/transformers/98aa65385e18b0efd17acd8bf64dcdf21406bb0c99c801c2d3c9f6bfd1f48f29.250d6dc755ccb17d19c7c1a7677636683aa35f0f6cb5461b3c0587bc091551a0.lock
Downloading: 100%|██████████| 718/718 [00:00<00:00, 603kB/s]
10/13/2020 07:58:23 - INFO - filelock -   Lock 140696551537464 released on /root/.cache/torch/transformers/98aa65385e18b0efd17acd8bf64dcdf21406bb0c99c801c2d3c9f6bfd1f48f29.250d6dc755ccb17d19c7c1a7677636683aa35f0f6cb5461b3c0587bc091551a0.lock
10/13/2020 07:58:23 - INFO - filelock -   Lock 140696551536624 acquired on /root/.cache/torch/transformers/f20f05d3ae37c4e3cd56764d48e566ea5adeba153dcee6eb82a18822c9c731

In [25]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/text-generation/run_generation.py

--2020-10-13 08:34:45--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/text-generation/run_generation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11046 (11K) [text/plain]
Saving to: ‘run_generation.py’

run_generation.py   100%[===================>]  10.79K  --.-KB/s    in 0s      

2020-10-13 08:34:45 (49.9 MB/s) - ‘run_generation.py’ saved [11046/11046]



In [26]:
%%bash
python run_generation.py --model_type gpt2 --model_name_or_path output --length 300 --prompt "Malfoy hadn’t noticed anything."

=== GENERATED SEQUENCE 1 ===
Malfoy hadn’t noticed anything. He was

still frowning, but had stopped smiling at Harry and Ron.

“Ah, Draco,” said Madame Maxime, throwing a brilliant

orange lantern onto the mantelpiece and speaking

toward Crabbe and Goyle. “Good — good. They’ll go

anywhere. Oh, the way they could make us look like

Irish flippers — ”

“They’re not Irish,” said Ron, thrusting a glittering

nod toward the boggart, who nodded and began to sing.

“Winky! Winky Winky has made a mistake!

Did you see the little old lady waving the badge?

Winky will not be supporting Gryffindor — ”

“Stop being a terrible commentator!” Ron hissed, pointing

his Firebolt at the boggart, which became a blur of

shadows.

“‘Neville will be cheering for his team!” shrieked Madam

Maxime as Malfoy and Malfoy’s champions soared to the

top of the hill and began to sprint toward the Firebolt,

both boys cackling.

“Ladies and gentlemen, six points to Gryffindor for first, please!”

smiled Madame 

2020-10-13 08:34:56.307943: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/13/2020 08:34:58 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
10/13/2020 08:35:19 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=300, model_name_or_path='output', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='', prompt='Malfoy hadn’t noticed anything.', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
